In [3]:
import sys
!{sys.executable} -m pip install --user sklearn-pandas

sys.path.append('/home/mluser/.local/lib/python3.5/site-packages')

In [4]:
import pandas as pd
import numpy as np
import sklearn.preprocessing, sklearn.decomposition, \
       sklearn.linear_model, sklearn.pipeline, sklearn.metrics, \
       sklearn.impute
from sklearn.feature_extraction.text import CountVectorizer
from sklearn_pandas import DataFrameMapper, cross_val_score

In [5]:
data = pd.DataFrame({'pet':      ['cat', 'dog', 'dog', 'fish', 'cat', 'dog', 'cat', 'fish'],
                     'children': [4., 6, 3, 3, 2, 3, 5, 4],
                     'salary':   [90., 24, 44, 27, 32, 59, 36, 27]})

In [6]:
data

,children,pet,salary
0,4.0,cat,90.0
1,6.0,dog,24.0
2,3.0,dog,44.0
3,3.0,fish,27.0
4,2.0,cat,32.0
5,3.0,dog,59.0
6,5.0,cat,36.0
7,4.0,fish,27.0


The mapper takes a list of tuples. The first is a column name from the pandas DataFrame, or a list containing one or multiple columns (we will see an example with multiple columns later). The second is an object which will perform the transformation which will be applied to that column. The third is optional and is a dictionary containing the transformation options, if applicable (see "custom column names for transformed features" below).

In [7]:
mapper = DataFrameMapper([
     ('pet', sklearn.preprocessing.LabelBinarizer()),
     (['children'], sklearn.preprocessing.StandardScaler())
])
np.round(mapper.fit_transform(data.copy()), 2)

array([[ 1.  ,  0.  ,  0.  ,  0.21],
       [ 0.  ,  1.  ,  0.  ,  1.88],
       [ 0.  ,  1.  ,  0.  , -0.63],
       [ 0.  ,  0.  ,  1.  , -0.63],
       [ 1.  ,  0.  ,  0.  , -1.46],
       [ 0.  ,  1.  ,  0.  , -0.63],
       [ 1.  ,  0.  ,  0.  ,  1.04],
       [ 0.  ,  0.  ,  1.  ,  0.21]])

The difference between specifying the column selector as 'column' (as a simple string) and ['column'] (as a list with one element) is the shape of the array that is passed to the transformer. In the first case, a one dimensional array will be passed, while in the second case it will be a 2-dimensional array with one column, i.e. a column vector.

Now that the transformation is trained, we confirm that it works on new data:

In [8]:
sample = pd.DataFrame({'pet': ['cat'], 'children': [5.]})
np.round(mapper.transform(sample), 2)

array([[1.  , 0.  , 0.  , 1.04]])

In certain cases, like when studying the feature importances for some model, we want to be able to associate the original features to the ones generated by the dataframe mapper. We can do so by inspecting the automatically generated transformed_names_ attribute of the mapper after transformation:

In [9]:
mapper.transformed_names_

['pet_cat', 'pet_dog', 'pet_fish', 'children']

We can provide a custom name for the transformed features, to be used instead of the automatically generated one, by specifying it as the third argument of the feature definition:

In [10]:
mapper_alias = DataFrameMapper([
    ('pet', sklearn.preprocessing.LabelBinarizer()),
    (['children'], sklearn.preprocessing.StandardScaler(), {'alias': 'children_scaled'})
])
_ = mapper_alias.fit_transform(data.copy())
mapper_alias.transformed_names_

['pet_cat', 'pet_dog', 'pet_fish', 'children_scaled']

By default the transformers are passed a numpy array of the selected columns as input. This is because sklearn transformers are historically designed to work with numpy arrays, not with pandas dataframes, even though their basic indexing interfaces are similar.

However we can pass a dataframe/series to the transformers to handle custom cases initializing the dataframe mapper with input_df=True:

In [11]:
from sklearn.base import TransformerMixin
class DateEncoder(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        dt = X.dt
        return pd.concat([dt.year, dt.month, dt.day], axis=1)

dates_df = pd.DataFrame({'dates': pd.date_range('2015-10-30', '2015-11-02')})
mapper_dates = DataFrameMapper([('dates', DateEncoder())], input_df=True)
mapper_dates.fit_transform(dates_df)

array([[2015,   10,   30],
       [2015,   10,   31],
       [2015,   11,    1],
       [2015,   11,    2]])

We can also specify this option per group of columns instead of for the whole mapper:

In [12]:
mapper_dates = DataFrameMapper([('dates', DateEncoder(), {'input_df': True})], df_out=True)
mapper_dates.fit_transform(dates_df)

,dates_0,dates_1,dates_2
0,2015,10,30
1,2015,10,31
2,2015,11,1
3,2015,11,2


By default the output of the dataframe mapper is a numpy array. This is so because most sklearn estimators expect a numpy array as input. If however we want the output of the mapper to be a dataframe, we can do so using the parameter df_out when creating the mapper:

In [13]:
mapper_df = DataFrameMapper([
    ('pet', sklearn.preprocessing.LabelBinarizer()),
    (['children'], sklearn.preprocessing.StandardScaler())
], df_out=True)
np.round(mapper_df.fit_transform(data.copy()), 2)

,pet_cat,pet_dog,pet_fish,children
0,1,0,0,0.21
1,0,1,0,1.88
2,0,1,0,-0.63
3,0,0,1,-0.63
4,1,0,0,-1.46
5,0,1,0,-0.63
6,1,0,0,1.04
7,0,0,1,0.21


Transformations may require multiple input columns. In these cases, the column names can be specified in a list:

In [14]:
mapper2 = DataFrameMapper([(['children', 'salary'], sklearn.decomposition.PCA(1))])
np.round(mapper2.fit_transform(data.copy()), 1)

array([[ 47.6],
       [-18.4],
       [  1.6],
       [-15.4],
       [-10.4],
       [ 16.6],
       [ -6.4],
       [-15.4]])

Multiple transformers can be applied to the same column specifying them in a list:

In [15]:
mapper3 = DataFrameMapper([(['age'], [sklearn.impute.SimpleImputer(),sklearn.preprocessing.StandardScaler()])])
data_3 = pd.DataFrame({'age': [1, np.nan, 3]})
mapper3.fit_transform(data_3)

array([[-1.22474487],
       [ 0.        ],
       [ 1.22474487]])

Only columns that are listed in the DataFrameMapper are kept. To keep a column but don't apply any transformation to it, use None as transformer:

In [16]:
mapper3 = DataFrameMapper([
    ('pet', sklearn.preprocessing.LabelBinarizer()),
    ('children', None)
])
np.round(mapper3.fit_transform(data.copy()))

array([[1., 0., 0., 4.],
       [0., 1., 0., 6.],
       [0., 1., 0., 3.],
       [0., 0., 1., 3.],
       [1., 0., 0., 2.],
       [0., 1., 0., 3.],
       [1., 0., 0., 5.],
       [0., 0., 1., 4.]])

A default transformer can be applied to columns not explicitly selected passing it as the default argument to the mapper:

In [17]:
mapper4 = DataFrameMapper([
    ('pet', sklearn.preprocessing.LabelBinarizer()),
    ('children', None)
], default=sklearn.preprocessing.StandardScaler())
np.round(mapper4.fit_transform(data.copy()), 1)

array([[ 1. ,  0. ,  0. ,  4. ,  2.3],
       [ 0. ,  1. ,  0. ,  6. , -0.9],
       [ 0. ,  1. ,  0. ,  3. ,  0.1],
       [ 0. ,  0. ,  1. ,  3. , -0.7],
       [ 1. ,  0. ,  0. ,  2. , -0.5],
       [ 0. ,  1. ,  0. ,  3. ,  0.8],
       [ 1. ,  0. ,  0. ,  5. , -0.3],
       [ 0. ,  0. ,  1. ,  4. , -0.7]])

Using default=False (the default) drops unselected columns. Using default=None pass the unselected columns unchanged.

Sometimes it is required to apply the same transformation to several dataframe columns. To simplify this process, the package provides gen_features function which accepts a list of columns and feature transformer class (or list of classes), and generates a feature definition, acceptable by DataFrameMapper.

For example, consider a dataset with three categorical columns, 'col1', 'col2', and 'col3', To binarize each of them, one could pass column names and LabelBinarizer transformer class into generator, and then use returned definition as features argument for DataFrameMapper:

In [18]:
from sklearn_pandas import gen_features
feature_def = gen_features(
    columns=['col1', 'col2', 'col3'],
    classes=[sklearn.preprocessing.LabelEncoder]
)
feature_def

[('col1', [LabelEncoder()]),
 ('col2', [LabelEncoder()]),
 ('col3', [LabelEncoder()])]

In [19]:
mapper5 = DataFrameMapper(feature_def)
data5 = pd.DataFrame({
    'col1': ['yes', 'no', 'yes'],
    'col2': [True, False, False],
    'col3': ['one', 'two', 'three']
})
mapper5.fit_transform(data5)

array([[1, 1, 0],
       [0, 0, 2],
       [1, 0, 1]])

If it is required to override some of transformer parameters, then a dict with 'class' key and transformer parameters should be provided. For example, consider a dataset with missing values. Then the following code could be used to override default imputing strategy:

In [20]:
feature_def = gen_features(
    columns=[['col1'], ['col2'], ['col3']],
    #classes=[{'class': sklearn.impute.SimpleImputer, 'strategy': 'most_frequent', 'missing_values': None}]
    classes=[{'class': sklearn.preprocessing.Imputer, 'strategy': 'most_frequent'}]
)
mapper6 = DataFrameMapper(feature_def)
data6 = pd.DataFrame({
    'col1': [None, 1, 1, 2, 3],
    'col2': [True, False, None, None, True],
    'col3': [0, 0, 0, None, None]
})
mapper6.fit_transform(data6)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[1., 1., 0.],
       [1., 0., 0.],
       [1., 1., 0.],
       [2., 1., 0.],
       [3., 1., 0.]])

In [27]:
mapper6 = DataFrameMapper([
    (['col1'], sklearn.impute.SimpleImputer(strategy='most_frequent')),
    (['col2'], sklearn.impute.SimpleImputer(strategy='most_frequent')),
    (['col3'], sklearn.impute.SimpleImputer(strategy='most_frequent')),
])
data6 = pd.DataFrame({
    'col1': [None, 1, 1, 2, 3],
    'col2': [True, False, None, None, True],
    'col3': [0, 0, 0, None, None]
})
mapper6.fit_transform(data6)

TypeError: ['col2']: unorderable types: NoneType() < bool()

DataFrameMapper supports transformers that require both X and y arguments. An example of this is feature selection. Treating the 'pet' column as the target, we will select the column that best predicts it.

In [103]:
from sklearn.feature_selection import SelectKBest, chi2
mapper_fs = DataFrameMapper([(['children','salary'], SelectKBest(chi2, k=1))])
mapper_fs.fit_transform(data[['children','salary']], data['pet'])

array([[90.],
       [24.],
       [44.],
       [27.],
       [32.],
       [59.],
       [36.],
       [27.]])